In [1]:
!date
import pandas as pd
import numpy as np
import sys
sys.path.append('../dev')

import utils

pd.options.display.max_colwidth = 100
import os

!cd ..

Fri Jul 14 11:14:04 PDT 2023


# Debugging ncHGT

Some pathways were enriched when they clearly should not have been. In this notebook, I identified and fixed the bug

Code from enrich.py

In [126]:
from scipy.stats import hypergeom
import sys
import os
import tqdm

pd.options.display.max_colwidth = 100

def get_sizes (data): #data= dataframe with gocam IDs and gene identifiers as columns
    return data['gocam'].value_counts()
    
def get_sets (gene_list):
    sets = []
    not_in_a_set = []
    members2setID = utils.csv2dict('../data/members2setID.csv')
    setID2members_input = {}
    for g in gene_list:
        s = members2setID.get(g)
        if s != None:
            sets = sets +s
            for i in s:
                if (i in setID2members_input) == False:
                    setID2members_input[i]={g}
                else:
                    prev = setID2members_input.get(i)
                    prev.add(g)
                    setID2members_input[i] = prev
        else:
            not_in_a_set.append(g)
    return not_in_a_set, list(set(sets)),setID2members_input #remove duplicates

def filter_gene_list(gene_list, Dict):
    filtered_gene_list = []
    filtered_out = []
    for gene in gene_list:
        if gene in Dict:
            filtered_gene_list.append(gene)
        else:
            filtered_out.append(gene)
    return filtered_out, filtered_gene_list

def count_genes(gene_list, Dict):
    gocam_counts = {} #key=gocam, value=list of genes in gocam that are also in the user's list
    for g in gene_list: 
            gocams = Dict.get(g)
            for gocam in gocams:
                if (gocam in gocam_counts) == False:
                    gocam_counts[gocam]=[g]
                else:
                    prev = gocam_counts.get(gocam)
                    prev.append(g)
                    gocam_counts[gocam] = prev
    return gocam_counts

#BENJAMINI HOCHBERG CORRECTION applied in correct_pval_and_format()
#ncHGT is either False (indicating that regular HGT should be done) or a positive integer denoting N for ncHGT
def hgt(counts, gocam_sizes, FDR, gene_list_size, background_gene_list_size, ncHGT = False):
    """ ncHGT is either False (for set or standard methods) or corresponds to N """
    results = []
    iterator = tqdm.tqdm(counts.items())
    for gocam, gene_list in iterator:
        count = len(gene_list) 
        gocam_size = gocam_sizes[gocam]
        pvalue = None
        if ncHGT:
            if count <=1: #avoid unnecessary calls to BiasedUrn due to computation time
                pvalue = 1
            else:
                pvalue = do_ncHGT(count -1,gocam,background_gene_list_size,ncHGT)
        else: #set or standard methods
            pvalue = hypergeom.sf(count-1, background_gene_list_size,  gocam_size, gene_list_size) 
        if pvalue < 1: #FDR:
            r = (gocam, pvalue, count, gocam_size, gene_list )
            results.append(r)
    return results

#Benjamini Hochberg correction
def correct_pval_and_format(enriched_gocams, background_num_gocams,show_significant,FDR):
    df = pd.DataFrame(enriched_gocams, columns =['url', 'pval (uncorrected)', '# entities in list','#entities in model','shared entities in gocam'])
    df.sort_values('pval (uncorrected)',inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['FDR_val'] = (df.index+1)*FDR/background_num_gocams
    df['Less_than'] = (df['pval (uncorrected)'] < df['FDR_val'])
    index = df.Less_than.where(df.Less_than==True).last_valid_index()
    df_significant = df
    if (show_significant):
        df_significant = df.loc[0:index].copy()
        if index == None:
            df_significant = pd.DataFrame(columns =['url', 'pval (uncorrected)', '# entities in list','#entities in model','shared entities in gocam'])
    df_display = df_significant[['url','pval (uncorrected)', '# entities in list', '#entities in model','shared entities in gocam']].copy()
    #modelID2title = pd.read_csv('../data/modelID2title_mouse.csv')
    temp = pd.read_csv('../data/modelID2title_mouse.csv',header = 0,names=['gocam','title'])
    modelID2title = pd.Series(temp.title.values,index=temp.gocam).to_dict()
    df_display['title'] = df_display['url'].map(modelID2title)
    cols = df_display.columns.to_list()
    cols[0]='title'
    cols[-1]='url'
    df_display = df_display[cols]
    return df_display

#Dict can only contain 1 instance of each gene per gocam (no duplicates)
#show_significant only affects the multiple testing correction. If the uncorrected pval > FDR, hgt() will already remove it
def enrich(gene_list, uni_list,uniprot2input,gocam_sizes, Dict, ncHGT=False, show_significant=True,FDR=.05):
    background_gene_list_size = len(Dict)
    if ncHGT: 
    #we consider the background size to be equal to the total # of genes 
    #(the sum of the weights of all entities would double count genes that occur in multiple sets
    #... is this the right thing to do though?
        background_gene_list_size = len(utils.csv2dict('../data/ID2gocam_mouse_ff.csv'))
        
    not_in_a_set, sets, setID2members_input_uni = get_sets(uni_list)
    
    setID2members_input = utils.map_dict_vals(uniprot2input, setID2members_input_uni)
    
    filtered_out1, set_list_filtered = filter_gene_list(sets,Dict)
    filtered_out2, gene_list_filtered = filter_gene_list(uni_list, Dict) #need to clean gene_list to only include genes in the gocam
    
    
    filtered_list = gene_list_filtered + set_list_filtered
    gene_list_size = len(filtered_list)
    
    flist2input = {**uniprot2input, **setID2members_input}
    filtered_list_as_genes = set(pd.Series(list(filtered_list)).map(flist2input).explode())
    filtered_out_genes = set(gene_list) - filtered_list_as_genes
    
    counts = count_genes(filtered_list, Dict)
    
    N_ncHGT = False
    if ncHGT == True:
        N_ncHGT = len(gene_list)-len(filtered_out_genes)
        if N_ncHGT <= 0:
            return "error no genes found in gocams"
        
    enriched_gocams = hgt(counts, gocam_sizes, FDR, gene_list_size, background_gene_list_size, ncHGT=N_ncHGT)
    background_num_gocams = len(gocam_sizes)
    df_display = correct_pval_and_format(enriched_gocams, background_num_gocams,show_significant,FDR)
    return filtered_out_genes, filtered_list, setID2members_input_uni, setID2members_input, df_display
    
def enrich_wrapper(filename, id_type, return_all = False, method = 'set', show_significant=True,FDR=.05,fpath= '../test_data', display_gene_symbol = True):
    """ returns (gene_list, filtered_out_genes, filtered_list, setID2members_input_uni, setID2members_input, df_display)"""
    #set method files
    gcs = '../data/gocam_sizes_mouse.csv'
    id2g = '../data/ID2gocam_mouse.csv'
    
    #standard method files
    if method == 'standard':
        gcs = '../data/gocam_sizes_mouse_ff.csv'
        id2g = '../data/ID2gocam_mouse_ff.csv'
    
    gene_list = pd.read_csv(os.path.join(fpath,filename),header=None,names = ['g'])
    gene_list_converted = []
    uniprot2input = {}
    not_converted = []
    
    #conversion to uniprot IDs not needed for a list of uniprot IDs
    if id_type == 'uniprot':
        gene_list_converted = gene_list.g
        uniprot2input = pd.Series(gene_list_converted.values,index=gene_list_converted).to_dict()
    else:
        gene_list_converted, uniprot2input, not_converted = utils.convert_IDs(gene_list,id_type)
    
    #read in dictionary and the gocam sizes
    x = pd.read_csv(gcs)
    gocam_sizes = pd.Series(x.sizes.values,index=x.gocam)
    Dict = utils.csv2dict(id2g)
    
    #call enrich()
    ncHGT = False
    if method == 'ncHGT':
        ncHGT = True
    #results: (filtered_out_genes, filtered_list, setID2members_input_uni, setID2members_input, df_display)
    results = enrich(list(gene_list.g), gene_list_converted, uniprot2input, gocam_sizes, Dict, ncHGT = ncHGT, show_significant = show_significant, FDR=FDR)
    
    if display_gene_symbol == True:
        results[4]['shared entities in gocam'] = utils.uniprot2gene(results[4]['shared entities in gocam'])
    if method == 'set' or method == 'ncHGT':
        print(f"Analysis run on {len(results[1])} entities from {len(gene_list)-len(results[0])} out of {len(gene_list)} input genes")
    if return_all:
        return (gene_list, *results)
    else:
        return results[4]

        
def compare2standard(d1, filename , symbol, FDR=.05,fpath = '../test_data/processed'):
    
    d2= enrich_wrapper(filename,symbol,method='standard',FDR = FDR,fpath = fpath)


    r1 = set(d1.title.values)
    r2 = set(d2.title.values)

    dif12 = set(r1)-set(r2)
    dif21 = set(r2)-set(r1)
        
    d2_all = enrich_wrapper(filename,symbol,method='standard',show_significant= False, FDR = FDR,fpath = fpath)
    dif12 = list(dif12)
    sizes = {}
    for i in dif12:
        s = d2_all[d2_all.title == i]["#entities in model"].values[0]
        sizes[i] = s

    df = d1[d1.title.apply(lambda x: x in dif12)].copy()
    if len(df) > 0:
        
        df['gene list size'] = df.title.apply(lambda x: sizes.get(x))
    
    print(f"Standard method yields {len(r2)} results, {len(dif21)} of which are unique")
    
    return df
        
        

Code from ncHGT.py

In [144]:
import rpy2
from rpy2.robjects.packages import importr
BiasedUrn = importr('BiasedUrn')

def get_M_wM():
    """ returns M, the number of entities in the background, and w_M, the mean size of entities in the background"""
    setID2members = utils.csv2dict('../data/setID2members.csv')
    l = []
    for s,m in setID2members.items():
        l.append(len(m))
    l = np.array(l)
    l = np.sort(l)
    num_empty_sets = np.sum(l==0)
    
    l = l[l!=0]
    mean = np.mean(l)#l[4:-4]) 1% trimmed mean?
    num_sets = len(l)
    bg = len(utils.csv2dict('../data/ID2gocam_mouse.csv'))
    M = bg-num_empty_sets
    
    w_M = np.round(((M-num_sets)+num_sets*mean)/M,decimals=2)
    return M, w_M

def make_initial_vectors(gocam2ID,setID2members, gc, M,w_M):
    """initializes counts vector (m) and weights vector (w), where each entity gets its own element in the arrays
- values in m only take on 0 (if there is no solo proteins) or 1
- values in w correspond to the weight of each element in m (weighted by the # genes in a set or 1 for solo proteins)"""
    w_gc = [1] #initialize with 1 as the weight of single proteins (irrespective of whether there are any)
    m_gc = [0] #initialize with 0 single proteins
    num_protein = 0
    for i in gocam2ID.get(gc):
        if "sset:" in i:
            w_i = len(setID2members.get(i))
            w_gc.append(w_i)
            m_gc.append(1)
        else:
            num_protein+=1
    m_gc[0] = num_protein
    m_gc.append(M-np.sum(m_gc)) #entities not in the gocam (roughly)
    w_gc.append(w_M) #weight for entities not in the gocam (all weighted as w_M (the mean))
    return w_gc, m_gc


def make_new_vectors(w_gc,m_gc,M,w_M):
    """compress the m and w vectors by grouping elements according to their weights
- w is the ordered set of unique weights for entities of the gocam + the background bin
- m[i] is the number of entities in the pathway with the weight specified in w[i] + the background bin"""
    w_temp = w_gc[:-1]
    if w_temp[0] != 1:
        print('Possible bug: w_temp[0] != 1',w_temp)
        
    w_new, m_temp = np.unique(w_temp, return_counts=True)
    m_temp[0]=m_gc[0] #w_gc and m_gc have weight 1 as w_gc[0] and the number of single proteins as m_gc[0]
    m_new = np.append(m_temp,np.array([M-np.sum(m_temp)]))
    w_new = np.append(np.unique(w_temp),np.array([w_M]))
    return w_new, m_new




def ncHGT_sf(XT,m,N,w):
    """survival function, sums PMF for all possibilities where K >= k by calling BiasedUrn"""
    #l = len(XT)/len(m)
    if len(XT) == 0:
        print('len(XT) = 0')
        return -1
    pval = 0
    #np.seterr(under='warn')
    for i in range(len(XT)):
        x = rpy2.robjects.IntVector(XT[i])
        pval = pval + BiasedUrn.dMFNCHypergeo(x,m,N,w, precision = 1e-10)[0]
    return pval


def enumerate_possibilities(m_new,i,prev_array):
    """enumerate all possible counts vectors"""
    first = True
    for j in range(m_new[i]+1):
        xt = prev_array.copy()
        xt[0][i] = j
        
        #recursion
        if (i < len(m_new)-1):
            xt = enumerate_possibilities(m_new, i+1, xt) #will return matrix (array of arrays)
            
        #combining results into matrix
        if not first:
            XT = np.concatenate([XT,xt], axis = 0)
        else:
            XT = xt
            first = False
    return XT


def do_ncHGT(k,gc,M,N):
    setID2members = utils.csv2dict('../data/setID2members.csv')
    gocam2ID = utils.csv2dict('../data/gocam2ID_mouse.csv')
    
    M, w_M = get_M_wM()
    
    #make weight (w) and bin size (m) vectors where each entity in the gocam gets its own entry
    w_in, m_in = make_initial_vectors(gocam2ID, setID2members, gc, M,w_M)
    
    #update m and w vectors by grouping sets of the same size
    w_new , m_new= make_new_vectors(w_in,m_in,M,w_M)

    #make XT matrix, an enumeration of all possible arangements of balls in bins based on m_new and w_new
    m_gc = m_new[:-1] #don't pass the background bin to XT
    XT = enumerate_possibilities(m_gc,0,np.zeros(shape=(1,len(m_gc))))
    
    #filter XT to only include the region of the sample space >= k (which is what we want to sum probabilities over)
    mask1 = (np.sum(XT, axis=1) >= k)
    XT = XT[mask1]

    #filter XT to ensure that more than N entities are not picked
    mask2 = (np.sum(XT, axis=1) <= N)
    XT = XT[mask2]

    #add the remaining entities to the m+1th bin (non gocam bin)
    x_mp1_vec = N- np.sum(XT, axis = 1) #number of balls to be drawn from the last bin (the non-gocam background)
    XT = np.concatenate((XT,x_mp1_vec.reshape(len(x_mp1_vec),1)), axis = 1)

    m = rpy2.robjects.IntVector(m_new)
    w = rpy2.robjects.FloatVector(w_new)
    pval = ncHGT_sf(XT,m,N,w)
    return pval



In [148]:
mac_comb_nc = enrich_wrapper('mac_comb.csv','Gene Symbol',method='ncHGT',show_significant = True,fpath = '../test_data/processed/')
mac_comb_nc#.query('title == "PI5P, PP2A and IER3 Regulate PI3K/AKT Signaling - Reactome"')

100%|█████████████████| 667/667 [03:10<00:00,  3.50it/s]


Analysis run on 620 entities from 511 out of 1519 input genes


,title,pval (uncorrected),# entities in list,#entities in model,shared entities in gocam,url
0,Resolution of Sister Chromatid Cohesion - Reactome,2.795107e-11,30,95,"[BIRC5, CDC20, KIF2C, CENPF, CENPA, NUF2, ZWILCH, AURKB, CENPN, SPC24, KNTC1, CENPK, PLK1, SKA1,...",http://model.geneontology.org/R-HSA-2500257
1,Interleukin-35 Signalling - Reactome,1.562032e-05,7,9,"[IL6ST, JAK2, STAT4, EBI3, STAT1, IL12A, sset:JAK1, JAK2, (TYK2)]",http://model.geneontology.org/R-HSA-8984722
2,Unwinding of DNA - Reactome,2.943151e-05,6,7,"[MCM5, MCM7, MCM2, MCM3, MCM4, MCM6]",http://model.geneontology.org/R-HSA-176974


In [142]:
mac_comb_nc.query('title == "Unwinding of DNA - Reactome"')

,title,pval (uncorrected),# entities in list,#entities in model,shared entities in gocam,url
2,Unwinding of DNA - Reactome,0.000029,6,7,"[MCM5, MCM7, MCM2, MCM3, MCM4, MCM6]",http://model.geneontology.org/R-HSA-176974


# Debugging

In [121]:
""" enrich wrapper #############################################################"""

#arguments
filename = 'mac_comb.csv'
id_type = 'Gene Symbol'
#filename = 'P97.csv'
#id_type = 'uniprot'
method='ncHGT'
FDR = 0.05
fpath = '../test_data/processed/'


#not an argument but I need to set this to true
ncHGT = True

#code
gcs = '../data/gocam_sizes_mouse.csv'
id2g = '../data/ID2gocam_mouse.csv'

gene_list = pd.read_csv(os.path.join(fpath,filename),header=None,names = ['g'])
gene_list_converted = []
uniprot2input = {}
not_converted = []

#conversion to uniprot IDs not needed for a list of uniprot IDs
if id_type == 'uniprot':
    gene_list_converted = gene_list.g
    uniprot2input = pd.Series(gene_list_converted.values,index=gene_list_converted).to_dict()
else:
    gene_list_converted, uniprot2input, not_converted = utils.convert_IDs(gene_list,id_type)

#read in dictionary and the gocam sizes
x = pd.read_csv(gcs)
gocam_sizes = pd.Series(x.sizes.values,index=x.gocam)
Dict = utils.csv2dict(id2g)

"""enrich ##########################################################################"""

#arguments
gene_list = list(gene_list.g)
uni_list = gene_list_converted



background_gene_list_size = len(Dict)
if ncHGT: 
#we consider the background size to be equal to the total # of genes 
#(the sum of the weights of all entities would double count genes that occur in multiple sets
#... is this the right thing to do though?
    background_gene_list_size = len(utils.csv2dict('../data/ID2gocam_mouse_ff.csv'))

not_in_a_set, sets, setID2members_input_uni = get_sets(uni_list)

setID2members_input = utils.map_dict_vals(uniprot2input, setID2members_input_uni)

filtered_out1, set_list_filtered = filter_gene_list(sets,Dict)
filtered_out2, gene_list_filtered = filter_gene_list(uni_list, Dict) #need to clean gene_list to only include genes in the gocam


filtered_list = gene_list_filtered + set_list_filtered
gene_list_size = len(filtered_list)

flist2input = {**uniprot2input, **setID2members_input}
filtered_list_as_genes = set(pd.Series(list(filtered_list)).map(flist2input).explode())
filtered_out_genes = set(gene_list) - filtered_list_as_genes

counts = count_genes(filtered_list, Dict)

N_ncHGT = False
if ncHGT == True:
    N_ncHGT = len(gene_list)-len(filtered_out_genes)
    if N_ncHGT <= 0:
        print("error no genes found in gocams")

background_num_gocams = len(gocam_sizes)


""" hgt #############################################################################################"""
#arguments
ncHGT = N_ncHGT

#code
results = []

#gocam = 'http://model.geneontology.org/R-HSA-1169408'
#gocam = 'http://model.geneontology.org/R-HSA-6811558'
gocam = 'http://model.geneontology.org/R-HSA-4641258'
gene_list = counts.get(gocam)

count = len(gene_list) 
gocam_size = gocam_sizes[gocam]
pvalue = None



pvalue = do_ncHGT(count -1,gocam,background_gene_list_size,ncHGT)
print(count -1,gocam,background_gene_list_size,ncHGT)
print('pvalue:',pvalue)


w_temp [1]
w_new: [1]
m_temp [47]
2 http://model.geneontology.org/R-HSA-4641258 5386 511
pvalue: 0.9883299115869503


In [112]:
N_ncHGT

511

In [138]:
#do_ncHGT(k,gc,M,N)

k = count-1
gc = gocam
M = background_gene_list_size
N = ncHGT

setID2members = utils.csv2dict('../data/setID2members.csv')
gocam2ID = utils.csv2dict('../data/gocam2ID_mouse.csv')

M, w_M = get_M_wM()

#make weight (w) and bin size (m) vectors where each entity in the gocam gets its own entry
w_in, m_in = make_initial_vectors(gocam2ID, setID2members, gc, M,w_M)

print('w in:',w_in)
print('m in:',m_in)
#update m and w vectors by grouping sets of the same size
w_new , m_new= make_new_vectors(w_in,m_in,M,w_M)
print('w new:',w_new)
print('m new:',m_new)
#make XT matrix, an enumeration of all possible arangements of balls in bins based on m_new and w_new
m_gc = m_new[:-1] #don't pass the background bin to XT
XT = enumerate_possibilities(m_gc,0,np.zeros(shape=(1,len(m_gc))))

#filter XT to only include the region of the sample space >= k (which is what we want to sum probabilities over)
mask1 = (np.sum(XT, axis=1) >= k)
XT = XT[mask1]

#filter XT to ensure that more than N entities are not picked
mask2 = (np.sum(XT, axis=1) <= N)
XT = XT[mask2]

#add the remaining entities to the m+1th bin (non gocam bin)
x_mp1_vec = N- np.sum(XT, axis = 1) #number of balls to be drawn from the last bin (the non-gocam background)
XT = np.concatenate((XT,x_mp1_vec.reshape(len(x_mp1_vec),1)), axis = 1)

m = rpy2.robjects.IntVector(m_new)
w = rpy2.robjects.IntVector(w_new)
pval = ncHGT_sf(XT,m,N,w)

w in: [1, 2.0]
m in: [47, 3911]
w new: [1. 2.]
m new: [  47 3911]


In [109]:
print(m)
print(m_new)
print(w)
print(w_new)

[1]    1 3957

[   1 3957]
[1] 1.000000 1.990652

[1.         1.99065184]


I found two bugs.
1. As a dum dum, I didn't think this through and 1.99 got converted to "1" as an R IntVector. I updated the code by rounding to two decimal places and also changing it from an IntVector to a FloatVector.
2. m_new was wrong. The number of single proteins was initially counted correctly, but in the "compression" step where the inital m was sorted and the number of sets of each size were counted, m_new[0] (corresponding to single proteins) was obtained by counting the number of instances of "1" in the initial, which was just once.

In [19]:
counts.get('http://model.geneontology.org/R-HSA-4641258')

['P28062', 'O75832', 'P28065']

In [4]:
import enrich

p97 = enrich.enrich_wrapper('P97.csv','uniprot',method='set',FDR = .05,fpath = '../test_data/processed/')
p97

100%|██████████████| 480/480 [00:00<00:00, 16320.38it/s]


Analysis run on 338 entities from 270 out of 766 input genes


,title,pval (uncorrected),# entities in list,#entities in model,shared entities in gocam,url
0,"PI5P, PP2A and IER3 Regulate PI3K/AKT Signaling - Reactome",0.000002,8,13,"[sset:PIP5K1A/B, sset:Activator:PI3K, sset:SYNJ/MTM(1), sset:Activated SRC,LCK,EGFR,INSR, sset:P...",http://model.geneontology.org/R-HSA-6811558
1,Unwinding of DNA - Reactome,0.000002,6,7,"[MCM3, MCM6, MCM4, MCM2, MCM7, MCM5]",http://model.geneontology.org/R-HSA-176974


In [6]:
mac_comb_nc.query('title == "PI5P, PP2A and IER3 Regulate PI3K/AKT Signaling - Reactome"')

,title,pval (uncorrected),# entities in list,#entities in model,shared entities in gocam,url
39,"PI5P, PP2A and IER3 Regulate PI3K/AKT Signaling - Reactome",0.000251,8,13,"[sset:PIP5K1A/B, sset:Activator:PI3K, sset:SYNJ/MTM(1), sset:Activated SRC,LCK,EGFR,INSR, sset:P...",http://model.geneontology.org/R-HSA-6811558
